In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset,DataLoader
import os

from sklearn.model_selection import KFold
from torch import nn
from torchsummary import summary
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


from torch.optim.lr_scheduler import StepLR
import pickle
from utils import * 

d:\programfilesssd2\miniconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\programfilesssd2\miniconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\programfilesssd2\miniconda\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
TARGET = "EDA"
with open('DICT_EDG300.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
dataX = loaded_data[TARGET]["dataX"]
dataY = loaded_data[TARGET]["dataY"]
len(dataX)

2760

In [3]:
len(dataX[0][0])

1200

In [4]:
# 十折交叉验证
kf = KFold(n_splits=5,shuffle=True,random_state=114514)
INDEX   = []
KFSORT = 1
for train_index, valid_index in kf.split(dataX):
    INDEX.append((train_index,valid_index))

# dataLoader_train = DataLoader(dataset_train, batch_size=32, shuffle=True,num_workers=4,pin_memory=True)
# dataLoader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=False,num_workers=4,pin_memory=True)

dataLoader_train = DataLoader(TensorDataset(dataX[INDEX[KFSORT][0]],dataY[INDEX[KFSORT][0]]), batch_size=128, shuffle=True,num_workers=4,pin_memory = True,prefetch_factor=8)
dataLoader_valid = DataLoader(TensorDataset(dataX[INDEX[KFSORT][1]],dataY[INDEX[KFSORT][1]]), batch_size=128, shuffle=False,num_workers=4,pin_memory = True,prefetch_factor=8)

print(f"载入数据，训练样本{len(dataX[train_index])}个，验证样本{len(dataY[valid_index])}个。")

载入数据，训练样本2208个，验证样本552个。


In [5]:
 len(dataY[valid_index])

552

In [6]:
modelName = "CNN"
target = "EDA300"

TRAIN_CNN(MODEL_CNN_EDA(input_channel=1),modelName =modelName,target=target,KFSORT=KFSORT,dataLoader_train=dataLoader_train,dataLoader_valid=dataLoader_valid,dataCount = len(dataX[train_index]),epochs=500,lr=0.0001,scheduler_step_size=100,scheduler_gamma=0.97)




epoch1: 19.9128	 acc_t[0.331]	 lr[0.0001]	 acc_val: 0.32971，	OK ====================	
epoch2: 19.7904	 acc_t[0.353]	 lr[0.0001]	 acc_val: 0.32971，	OK ====================	
epoch3: 19.7471	 acc_t[0.359]	 lr[0.0001]	 acc_val: 0.38768，	OK ====================	
epoch4: 19.7738	 acc_t[0.345]	 lr[0.0001]	 acc_val: 0.38768，	OK ====================	
epoch5: 19.6944	 acc_t[0.379]	 lr[0.0001]	 acc_val: 0.41304，	OK ====================	
epoch6: 19.6635	 acc_t[0.392]	 lr[0.0001]	 acc_val: 0.41666，	OK ====================	
epoch7: 19.7415	 acc_t[0.365]	 lr[0.0001]	 acc_val: 0.41666，	OK ====================	
epoch8: 19.7296	 acc_t[0.371]	 lr[0.0001]	 acc_val: 0.40036，	-1.63	
epoch9: 19.6511	 acc_t[0.375]	 lr[0.0001]	 

KeyboardInterrupt: 

In [ ]:

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:582"

In [ ]:
# 0 43
# 60 42



x = range(0,epochs) # x轴坐标值113  0.00009 -> 81
plt.plot(trainAccHis,c = 'r') # 参数c为color简写，表示颜色,r为red即红色

In [ ]:
x = range(0,epochs) # x轴坐标值
plt.plot(lossHistory,c = 'r') # 参数c为color简写，表示颜色,r为red即红色

In [ ]:

x = range(0,epochs) # x轴坐标值
plt.plot(accHistory,c = 'r') # 参数c为color简写，表示颜色,r为red即红色


In [ ]:

with open("loss_"+model_name+"_"+TARGET+"_5("+str(KFSORT+1)+").pkl", 'wb') as file:
    pickle.dump(dict({"loss":lossHistory,"acc":accHistory,"trainAccHis":trainAccHis}), file)

In [ ]:
STOP
model = MODEL_CNN_EDA(input_channel=1)
model = model.to(device)

epochs = 500
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #0.0008

criterion = torch.nn.CrossEntropyLoss()

scheduler = StepLR(optimizer, step_size=100, gamma=0.97)

lossHistory = []
accHistory = []
trainAccHis = []



early_stopping = EarlyStopping(patience=1200,filename="CNN_"+TARGET+"300_5("+str(KFSORT+1)+").pth")

for epoch in range(0,epochs):
    model.train()
    LOSS = 0
    ACC = 0
    for batch_data, batch_targets in dataLoader_train:
        optimizer.zero_grad()
        outputs = model(batch_data.to(device))
        loss = criterion(outputs, batch_targets.to(device))
        loss.backward()
        
        LOSS += loss.detach().cpu().numpy()
        ACC += (torch.max(outputs,dim=1)[1] == batch_targets.to(device)).sum()
        
        optimizer.step()
        
    scheduler.step()
    
    lossHistory.append(LOSS)
    
    print(f"epoch{epoch+1}: {str(LOSS)[0:7]}",end="\t ")
    cal = (ACC/dataCount).cpu()
    print(f"acc_t[{str(np.array(cal))[0:5]}]",end="\t ")
    trainAccHis.append(cal)
    print("lr["+str(optimizer.param_groups[0]['lr'])+"]",end="\t ")
    
#     for p in model.parameters():
#         print(p.grad.data)
    
    model.eval()
    with torch.no_grad():
        all_outputs = []
        all_targets = []
        for batch_data, batch_targets in dataLoader_valid:
            outputs = model(batch_data.to(device))
            _, predicted = torch.max(outputs.data, 1)
            all_outputs.append(predicted.cpu().numpy())
            all_targets.append(batch_targets.cpu().numpy())

        all_outputs = np.concatenate(all_outputs)
        all_targets = np.concatenate(all_targets)
        accuracy = accuracy_score(all_targets, all_outputs)
        accHistory.append(accuracy)
#         accAverHis.append(np.mean(np.array(accHistory)))
        print(f"acc_val: {str(accuracy)[0:7]}，",end="\t")
        print('OK '+'='*20 if(np.max(np.array(accHistory)) == accuracy) else str(100*(accuracy - np.max(np.array(accHistory))))[0:5],end="\t")
    
    
    if(epoch>50):
        print(f"[{ str(np.abs(np.mean(lossHistory)-np.mean(lossHistory[0:epoch-50])))[0:4] }")
#         if(np.abs(np.mean(lossHistory)-np.mean(lossHistory[0:epoch-50]))<0.05):
#             if(np.abs(100*(accuracy - np.max(np.array(accHistory))))<5):
#                 print("已提前结束。")
#                 break

                
    else:
        print("")
                    
    early_stopping(accuracy, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break